# Legislación

## Contexto
Los cerros que rodean la planicie central de la cuenca de Santiago imponen fuertes restricciones a la circulación de vientos y, por ende, a la renovación del aire al interior de la cuenca. Por ello, en épocas de estabilidad atmosférica los contaminantes quedan atrapados dentro de la cuenca que alberga a la ciudad de Santiago.

Por esta situación en particular santiago se ha visto varias veces enfrentado con una contaminación muy alta, llegando a ser declarada zona saturada por Ozono, Material Particulado, Monoxido de Carbono y Dioxido de Nitrógeno en 1996, el año anterior se ordena la creación del _Plan de Prevención y Descontaminación Atmosférica_ de la Región Metropolitana, que desde 1998 norma con el objetivo de:

El PPDA tiene por objetivo cumplir con las normas de calidad de aire cuyo propósito principal es la protección de la salud de la población. Ésta se encuentra habitualmente expuesta a concentraciones de contaminantes que afectan su bienestar físico y su calidad de vida en general. -- DS N°16 de 1998, Minsegpres, CONAMA

Con esto se realizan desde el mismo año mediciones diarias de distintos agentes contaminantes para evaluar tanto la situación actual del aire en santiago como para medir el rendimiento del PPDA.

| Fecha        | Tipo    | Nombre                                 |
|--------------|---------|----------------------------------------|
| 26-oct-95    | Decreto | Procedimiento PPDA                     |
| 01-ago-96    | Decreto | Zona saturada por O3 MP10 MP2,5 CO NO2 |
| 06-jun-98    | Decreto | Establece PPDA                         |
| 29-ene-04    | Decreto | Actualiza PPDA                         |
| 16-abr-10    | Decreto | Actualiza PPDA                         |
| 15-nov-14    | Decreto | Zona Saturada MP2,5                    |
| 24-nov-17    | Decreto | Establece PPDA                         |


Aquí intentaremos responder si estas legislaciones y regulaciones tienen algún efecto real sobre la disminución de la contaminación en la cuenca de Santiago.

In [90]:
# Imports used on the script

import matplotlib
matplotlib.use('tkagg')

import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

In [91]:
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])
# fit to an order-3 polynomial data
x = np.arange(5)
y = 3 - 2 * x + x ** 2 - x ** 3
model = model.fit(x[:, np.newaxis], y)
model.named_steps['linear'].coef_

array([ 3., -2.,  1., -1.])

In [92]:
# Import climate data
df = pd.read_csv("C:/Users/basty/Documents/intro_mineria_dcc_2019/Proyecto/ClimaYEmisionesData/DatosHito1CleanWithDates.csv")

# Example of how the table looks
df.head()

,Fecha,anno,mes,dia,O3,MP10,CO,SO2,MP25,NO2,Tmedia,Tmax,Precipitaciones
0,19970402,1997,4,2,18.8750,111.565,NaN,NaN,NaN,NaN,17.6,27.5,0.0
1,19970403,1997,4,3,14.2083,141.333,NaN,NaN,NaN,NaN,16.0,27.2,0.0
2,19970404,1997,4,4,30.4545,156.950,NaN,NaN,NaN,NaN,17.6,31.1,0.0
3,19970405,1997,4,5,NaN,0.000,NaN,NaN,NaN,NaN,16.9,29.6,0.0
4,19970406,1997,4,6,NaN,0.000,NaN,NaN,NaN,NaN,15.2,25.6,0.0


In [93]:
# We split the database on periods according to different versions of PPDA program

prePPDA = df[(df['Fecha']<19980606)]

primerPPDA = df[(df['Fecha']>=19980606) & (df['Fecha']<=20040129)]

segundoPPDA = df[(df['Fecha']>=20040129) & (df['Fecha']<=20100416)]

# This one is a complete new DS that declares the RM a MP2.5 Saturation Zone
tercerPPDA = df[(df['Fecha']>=20100416) & (df['Fecha']<=20171124)]

cuartoPPDA = df[(df['Fecha']>=20171124)]

In [98]:
model = Pipeline([('poly', PolynomialFeatures(degree=4)),
                  ('linear', LinearRegression(fit_intercept=False))])

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(prePPDA)

model = model.fit(imp.transform(prePPDA),imp.transform(prePPDA))
model.named_steps['linear'].coef_

array([[-5.66605813e-17, -1.07557030e-22,  9.98346203e-24, ...,
         1.16725096e-27, -3.23876384e-27, -2.85158129e-26],
       [-8.49733872e-21, -1.61414426e-26,  1.50011590e-27, ...,
         1.74977241e-31, -4.85584308e-31, -4.27435306e-30],
       [ 2.91361854e-19,  5.53746792e-25, -5.21332743e-26, ...,
        -6.00878633e-30,  1.66282403e-29,  1.46430793e-28],
       ...,
       [-2.90546213e-19, -5.38560876e-25,  6.24339122e-26, ...,
         5.84269341e-30, -1.72036936e-29, -1.48995783e-28],
       [ 2.87338861e-19,  6.37708802e-25, -8.02008956e-28, ...,
        -5.31796260e-30,  1.52177949e-29,  1.34382144e-28],
       [ 2.04935996e-18,  3.99506460e-24, -3.33508156e-25, ...,
        -4.17389407e-29,  1.13763695e-28,  1.00446160e-27]])

In [ ]:
# Plot outputs
plt.scatter(prePPDA['Fecha'], prePPDA['MP10'],  color='black')
plt.title('Test Data')
plt.xlabel('Fecha')
plt.ylabel('ug/m^3')
plt.xticks(())
plt.yticks(())

plt.show()